In [ ]:
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

In [ ]:
%matplotlib inline

from pathlib import Path
import numpy as np
import pandas as pd
import pandas_datareader.data as web
import gc
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
from sklearn.preprocessing import scale
import talib
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Define the path
DATA_STORE = Path('/home/sayem/Desktop/Project/data/dataset.h5')

with pd.HDFStore(DATA_STORE) as store:
    keys = store.keys()
    
# Print keys in desired format
print("[", end="")
for i, key in enumerate(keys):
    if i < len(keys) - 1:
        print(f"'{key}', ", end="")
    else:
        print(f"'{key}'", end="")
print("]")

In [ ]:
with pd.HDFStore(DATA_STORE) as store:
    keys = store.keys()
    data = store[keys[0]]

In [ ]:
data.info()

In [ ]:
STOP

In [ ]:
DATA_STORE = Path('/home/sayem/Desktop/Project/data/assets.h5')

top = 250

with pd.HDFStore(DATA_STORE) as store:
    data = store[f'data/top{top}_dataset']

In [ ]:
selected_data = data[['open', 'high', 'low', 'close', 'volume']]

In [ ]:
selected_data.info()

In [ ]:
selected_data

In [ ]:
import warnings
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
from tqdm import tqdm
import shap
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.pipeline import Pipeline
import os
import gc

# Filter out warning messages
warnings.filterwarnings('ignore')

# Set pandas display options
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

# Set seaborn style
sns.set_style('whitegrid')

# # Add the parent directory to sys.path
# sys.path.insert(1, os.path.join(sys.path[0], '..'))

# Index and deciles for data slicing
idx = pd.IndexSlice
deciles = np.arange(.1, 1, .1).round(1)

In [ ]:
import pandas as pd
from pathlib import Path

DATA_STORE = Path('data/250_dataset.h5')

with pd.HDFStore(DATA_STORE) as store:
    dataset = store['/data/YEAR_20220906_20230811']

In [88]:
import pandas as pd
import numpy as np

def rank_and_quantize(df, TARGET_col='TARGET_ret_fwd_frac_order'):
    """
    Ranks stocks within each date based on the specified TARGET column and then bucket them into quantiles.
    
    Parameters:
        df (pd.DataFrame): Input dataframe with MultiIndex (date, ticker).
        TARGET_col (str): Column name based on which the ranking needs to be done.
    
    Returns:
        pd.DataFrame: Dataframe with original column, an additional column for ranks, and quantized values.
    """
    rank_col_name = TARGET_col + '_rank'
    quant_col_name = TARGET_col + '_quantiled'
    
    # Ranking stocks within each date so that the highest value gets rank 1 (considered best)
    df[rank_col_name] = df.groupby('date')[TARGET_col].rank(method="average", ascending=False).astype(int) # Change to descending

    # Bucketing the ranks into quantiles such that rank 1 is in the uppermost bucket (labelled as 1.0)
    quantile_labels = [1.0, 0.75, 0.5, 0.25, 0.0]
    df[quant_col_name] = pd.qcut(df[rank_col_name], q=5, labels=quantile_labels).astype(float)
    
    # # Sorting by MultiIndex levels to preserve the original structure
    # df.sort_index(level=['date', 'ticker'], ascending=[True, True], inplace=True)
    
    return df

# # Sample usage:
# tickers = ['AAPL', 'GOOGL', 'AMZN', 'MSFT', 'TSLA']
# dates = pd.date_range(start='2023-01-01', periods=10)
# arrays = [dates, tickers]
# index = pd.MultiIndex.from_product(arrays, names=('date', 'ticker'))
# dummy_data = np.random.rand(len(index)).astype(np.float32)
# df = pd.DataFrame({'TARGET_ret_fwd_frac_order': dummy_data}, index=index)

# Now using your dataset:
quantized_df = rank_and_quantize(dataset.copy())
cols = ['TARGET_ret_fwd_frac_order', 'TARGET_ret_fwd_frac_order_rank', 'TARGET_ret_fwd_frac_order_quantiled']
quantized_df[cols]

TARGET_ret_fwd_frac_order  TARGET_ret_fwd_frac_order_rank  \
ticker date                                                                    
AA     2022-09-06                   6.518137                             171   
       2022-09-07                   7.983007                             153   
       2022-09-08                  11.608492                             126   
       2022-09-09                  10.432384                             142   
       2022-09-12                   8.142491                             111   
...                                      ...                             ...   
ZTS    2023-08-07                  24.922548                              59   
       2023-08-08                  21.726665                              73   
       2023-08-09                  20.455498                              78   
       2023-08-10                  21.730034                              75   
       2023-08-11                  21.730034                              76   

                   TARGET_ret_fwd_frac_order_quantiled  
ticker date                                             
AA     2022-09-06                                 0.25  
       2022-09-07                                 0.50  
       2022-09-08                                 0.50  
       2022-09-09                                 0.50  
       2022-09-12                                 0.50  
...                                                ...  
ZTS    2023-08-07                                 0.75  
       2023-08-08                                 0.75  
       2023-08-09                                 0.75  
       2023-08-10                                 0.75  
       2023-08-11                                 0.75  

[59607 rows x 3 columns]

In [92]:
# Suppose you want to see values for the date '2023-01-01'
idx = pd.IndexSlice

specific_date = '2023-08-07'

# Extract rows corresponding to the specific date
date_df = quantized_df.loc[idx[:, specific_date], :]

# Now, check value counts for the desired column
value_counts = date_df['TARGET_ret_fwd_frac_order_quantiled'].value_counts()

print(value_counts)


0.00    51
0.50    51
1.00    51
0.75    51
0.25    50
Name: TARGET_ret_fwd_frac_order_quantiled, dtype: int64
